# Section 05: Data Cleaning in Pandas

1. Map and Apply
2. Groupby
3. Joining DataFrames
4. Missing Values

## `.map()` and `.apply()`

These are two methods used to run functions across multiple rows/columns of your DataFrame, and there are multiple ways to use each.

In [1]:
import numpy as np
import pandas as pd

In [4]:
car_df = pd.read_csv('auto-mpg.csv', na_values='?')

In [5]:
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


In [6]:
car_df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


### a) Broadcasting simple operations

i.e. convert `year` into its full year by adding `1900` to each value (we're assuming that the year range of 70-82 is in the 1900s)

In [8]:
car_df['model year'] = car_df['model year'] + 1900 # not destructive, we'd have to reassign the new values

### b) Boolean masking

(also useful to understand this for filtering dataframes)

In [9]:
car_df[['mpg', 'cylinders', 'displacement']] % 2 == 0

,mpg,cylinders,displacement
0,True,True,False
1,False,True,True
2,True,True,True
3,True,True,True
4,False,True,True
...,...,...,...
387,False,True,True
388,True,True,False
389,True,True,False
390,True,True,True


In [10]:
car_df['weight'] > 2800 # outputs a series of booleans
# showing whether each value meets the condition set

0       True
1       True
2       True
3       True
4       True
       ...  
387    False
388    False
389    False
390    False
391    False
Name: weight, Length: 392, dtype: bool

We can use this to create new, perhaps more useful columns!

In [11]:
car_df['heavy'] = car_df['weight'] > 2800

In [12]:
car_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,heavy
0,18.0,8,307.0,130,3504,12.0,1970,1,chevrolet chevelle malibu,True
1,15.0,8,350.0,165,3693,11.5,1970,1,buick skylark 320,True
2,18.0,8,318.0,150,3436,11.0,1970,1,plymouth satellite,True
3,16.0,8,304.0,150,3433,12.0,1970,1,amc rebel sst,True
4,17.0,8,302.0,140,3449,10.5,1970,1,ford torino,True


### c. The `.map()` function

`.map()` only works on Series, or on single columns!

You can supply a condition directly into the lambda, or define specific functions.

In [14]:
car_df['model year'].map(lambda y: y < 1980)

0       True
1       True
2       True
3       True
4       True
       ...  
387    False
388    False
389    False
390    False
391    False
Name: model year, Length: 392, dtype: bool

In [17]:
# tangent - for car names - let's see what names exist
# unique(), nunique(), value_counts()
car_df['car name']

0      chevrolet chevelle malibu
1              buick skylark 320
2             plymouth satellite
3                  amc rebel sst
4                    ford torino
                 ...            
387              ford mustang gl
388                    vw pickup
389                dodge rampage
390                  ford ranger
391                   chevy s-10
Name: car name, Length: 392, dtype: object

In [18]:
car_df['car name'].map(lambda n: 'ford' in n)

0      False
1      False
2      False
3      False
4       True
       ...  
387     True
388    False
389    False
390     True
391    False
Name: car name, Length: 392, dtype: bool

In [19]:
# for non-Boolean outputs, you can define a function to feed into the lambda

def car_brand(value):
    if 'ford' in value:
        return 'Ford'
    elif 'buick' in value:
        return 'Buick'
    elif 'chevrolet' in value:
        return 'Chevrolet'
    else:
        return 'Other'
    

In [20]:
car_df['car name'].map(lambda n: car_brand(n))

0      Chevrolet
1          Buick
2          Other
3          Other
4           Ford
         ...    
387         Ford
388        Other
389        Other
390         Ford
391        Other
Name: car name, Length: 392, dtype: object

### d. The `.apply()` function

This works on a DataFrame, or on multiple columns of your DataFrame.


Before that, `applymap()`, **a different method**, is like `.map()` but you can do functions across multiple columns or on an entire DataFrame. 


i.e. turning all your data into strings

In [22]:
car_df.applymap(lambda value: str(value)) # ['displacement']

# remember this isn't destructive until you reassign it to the variable!

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,heavy
0,18.0,8,307.0,130,3504,12.0,1970,1,chevrolet chevelle malibu,True
1,15.0,8,350.0,165,3693,11.5,1970,1,buick skylark 320,True
2,18.0,8,318.0,150,3436,11.0,1970,1,plymouth satellite,True
3,16.0,8,304.0,150,3433,12.0,1970,1,amc rebel sst,True
4,17.0,8,302.0,140,3449,10.5,1970,1,ford torino,True
...,...,...,...,...,...,...,...,...,...,...
387,27.0,4,140.0,86,2790,15.6,1982,1,ford mustang gl,False
388,44.0,4,97.0,52,2130,24.6,1982,2,vw pickup,False
389,32.0,4,135.0,84,2295,11.6,1982,1,dodge rampage,False
390,28.0,4,120.0,79,2625,18.6,1982,1,ford ranger,False


In [23]:
car_df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name', 'heavy'],
      dtype='object')

`.apply()` allows us to perform specific functions that take different columns as inputs. For example, if we want to engineer a new variable that's some combination of other variables. I usually define the function I want to use in `.apply()` taking in a row as its argument.


Let's create a variable that finds the horsepower per weight of the car.

In [ ]:
def horsepower_per_weight(row):
    return row['horsepower']/row['weight']

In [ ]:
car_df.apply(lambda r: horsepower_per_weight(r), axis=1) # axis = 1 is for columns

## `groupby()` methods for aggregation

To get aggregated information about subsets of your data. Pretty much every time you use a groupby, you have to pair it with an aggregation function.


Some common aggregations:
* `.min()`: returns the minimum value for each column by group  
* `.max()`: returns the maximum value for each column by group  
* `.mean()`: returns the average value for each column by group  
* `.median()`: returns the median value for each column by group  
* `.count()`: returns the count of each column by group

In [ ]:
car_df.head()

In [ ]:
car_df.groupby('cylinders').mean()

## Combining DataFrames

### `pd.concat()` and `pd.join()`

`pd.concat` stacks DataFrames together while maintaining columns or index.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])


df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'], 
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
pd.concat([df1, df2, df3])

In [ ]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                    index=[2, 3, 6, 7])
df4

In [ ]:
df1

In [ ]:
pd.concat([df1, df4], join='inner', axis=1) 

You can supply the `join` argument to `pd.concat`, but Pandas also has a `.join()` method that can be used interchangably in most cases.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join


There are 4 main types of joins: inner, left, right and outer. This is the same kind of join that SQL uses! https://www.w3schools.com/sql/sql_join.asp

- An inner join only keeps the rows that have indexes in both tables.
- Left join keeps all rows in the left table and supplements that information with info from the right table
- Right join does the opposite of a left join
- Outer join returns a combination of all rows

Also, `pd.merge` exists: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge

In [ ]:
pd.concat([df1, df4], join='outer', axis=1) 

In [ ]:
df1.join(df4, how='left', rsuffix='4') # why do we need the rsuffix???

## Missing Data

Definitely read up on the different ways you can go about filling in missing data.

`df.isna()`, `df.fillna()`

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [ ]:
test = pd.concat([df1, df4], join='outer', axis=1).drop(['B', 'D'], axis=1)
test

In [ ]:
test.isna().sum()

In [ ]:
test.fillna(0)

In [ ]:
# filling NaNs per column
na_values_per_column = {'A': 3, 'C': 2, 'F': 4}
test.fillna(value=na_values_per_column)

## For 1x1s:

- Attempt `Project: Data Cleaning - Lab`